In [77]:
%load_ext autoreload
%autoreload 2

from notebook import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# # Copie les notebooks et supprime les sorties.
# copy_and_clean_notebooks()

# Téléchargement des dialogues

In [23]:
env = AppInsightsAPIEnv(".env")

env.APP_INSIGHTS_API_ID

'7706f68b-7ff7-461c-86e0-f66ad9d4d030'

In [24]:
start_dt = "2021-12-21T10:54:54Z"
end_dt = "2021-12-21T11:54:54Z"

In [25]:
data = get_tn_dialogs(env, start_dt, end_dt)
data.shape

(40, 5)

In [26]:
data.head()

,main_dialog_uuid,timestamp,author,session_id,text
0,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:26.945023+00:00,p10-chatbot-bot,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,What can I help you with today?
1,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:28.479733+00:00,You,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,I want to go to Paris from London tomorrow and...
2,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:29.333668+00:00,p10-chatbot-bot,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,Please confirm the following information:\n- Y...
3,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:30.250325+00:00,You,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,Non
4,5cdf2384-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:32.158274+00:00,p10-chatbot-bot,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,What else can I do for you?


# Analyse des insatisfactions

In [27]:
analysis_name = "2021_12_21_analysis"

In [78]:
analyser = InsatisfactionsAnalyser(data)
analyser.load(PARQUET_PATH, analysis_name)
analyser.display()

In [55]:
analyser.save(PARQUET_PATH, analysis_name)

# Enregistrement du rapport sur Github

## Distribution des erreurs

In [83]:
print("## Distribution des erreurs\n")

tmp = analyser.res["error_type"].value_counts().to_frame("error_nb")
tmp["%"] = analyser.res["error_type"].value_counts(normalize=True)
tmp.index.name = "error_type"

print(tmp.to_html(float_format=lambda x: f"{x:0.2f}"))

## Distribution des erreurs

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>error_nb</th>
      <th>%</th>
    </tr>
    <tr>
      <th>error_type</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th></th>
      <td>7</td>
      <td>0.78</td>
    </tr>
    <tr>
      <th>luis</th>
      <td>1</td>
      <td>0.11</td>
    </tr>
    <tr>
      <th>chatbot</th>
      <td>1</td>
      <td>0.11</td>
    </tr>
  </tbody>
</table>


## Commentaires

In [75]:
print("## Commentaires\n")

for comment in analyser.res["comment"]:
    if comment:
        print(comment)

## Commentaires

sdfsdf
dsfsdf


## Utterrances pour LUIS

In [76]:
print("## Utterrances pour LUIS\n")

for utterances in analyser.res["utterances"]:
    if utterances:
        print(utterances)

## Utterrances pour LUIS

I want to go to Paris from London tomorrow and I will stay 3 days. I have 120 dollars.
What else can I do for you?
